# Customer Segmentaton of Retail Data


### Problem Statement

It is a critical requirement for business to understand the value derived from a customer. RFM is a method used for analyzing customer value.

Customer segmentation is the practice of segregating the customer base into groups of individuals based on some common characteristics such as age, gender, interests, and spending habits.

Our objective here is to perform customer segmentation using RFM analysis. The resulting segments can be ordered from most valuable (highest recency, frequency, and value) to least valuable (lowest recency, frequency, and value).


In [1]:
__author__ = "Ovo Okpubuluku"
__email__ = "ovo@live.ca"

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime

## Now we create a Data Class to ingest the data and show us some summary statistics

In [3]:
class Data:
    def __init__(self, dataset):
        
        '''loading dataframes'''
        
        self.dataset = dataset
        self.process_data()
        
    def process_data(self):
        self._create_df()
        self._column_info()
        self._print_df_stats()
        self._check_duplicates()
       
    def _create_df(self):
        '''loads and prepares dataframe'''
        self.df = self._load_dataset(dataset)        
        
    def _column_info(self):
        self.cat_cols = self._cat_cols(self.df)
        self.num_cols = self._num_cols(self.df)
        
    def _print_df_stats(self):
        print('  \n ----------Train Data Info---------')
        self.printstats(self.df)
        self._check_nan(self.df)
    
    def _load_dataset(self, file):
        return pd.read_excel(file)
    
    def printstats(self, df):
        print('---------------------------------------------------------')
        print('Shape of Dataframe - {}'.format(df.shape))
        print('---------------------------------------------------------')
        print('\n Dataframe Info: \n')
        print('n{}'.format(df.info()))
        print('---------------------------------------------------------')
        print(' Categorical Features Stats: \n \n{}'.format(df.describe(include='O')))
        print('-------------------------------------------------')
        print(' Numerical Features Stats:- \n \n{}'.format(df.describe()))
              
    def _check_nan(self, df):
        '''Checks and verifies presence of null values in Dataframe'''
        nan = np.sum(df.isna().sum())
        if nan == 0:
            print('\n\n : There are no null values in the dataframes')
        else:
            print('The following columns have null values\n\n{}'.format(df.isnull().sum()))
              
    def _cat_cols(self, df):
        '''finds and lists Categorical Columns in Dataframe'''
        self.cat_cols = df.select_dtypes(include=['O']).columns.tolist()
        print('Categorical Columns list: {}'.format(self.cat_cols))
        print('---------------------------------------------------------------------')
        return self.cat_cols
              
    def _num_cols(self, df):
        '''finds and lists Numerical Columns in Dataframe'''
        self.num_cols = df.select_dtypes(exclude=['O']).columns.tolist()
        print('Numerical Columns list: {}'.format(self.num_cols))
        print('---------------------------------------------------------------------')
        return self.num_cols
              
              
    def _check_duplicates(self):
        '''Checks presence of duplicate entries'''
        print('\n : There are {} duplicate values in the Dataframe'.format(self.df.duplicated().sum()))

In [4]:
dataset = "./online_retail.xlsx"

data = Data(dataset)

Categorical Columns list: ['InvoiceNo', 'StockCode', 'Description', 'Country']
---------------------------------------------------------------------
Numerical Columns list: ['Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID']
---------------------------------------------------------------------
  
 ----------Train Data Info---------
---------------------------------------------------------
Shape of Dataframe - (541909, 8)
---------------------------------------------------------

 Dataframe Info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null 

In [5]:
data.df.Quantity[data.df.Quantity < 0].count()

10624

In [6]:
data.df.UnitPrice[data.df.UnitPrice < 0].count()

2

In [7]:
data.df['CustomerID'].nunique()

4372

In [41]:
print('The first order date in the data is: ')
print(data.df['InvoiceDate'].min())

print('The last order date in the data is: ')
print(data.df['InvoiceDate'].max())

The first order date in the data is: 
2010-12-01 08:26:00
The last order date in the data is: 
2011-12-09 12:50:00


In [8]:
print(data.df.shape)
data.df.head()

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Handling null values, Duplicate values, and Unrealistic values from Dataframe

1. We notice from the information above that we have 135080 null values for customerID. We will have to remove all the entries without customerID because we cannot perform segmentation and cohort analysis on Customers that we cannot identify.

2. The 'Quantity' and 'UnitPrice' features have negative numbers. We will have to remove these numbers, as they are not realistic. They are probably returns or reversed transactions that were not concluded.


3. We also notice that we have 1454 null values for Description. This column has no real effect on the data, and we may end up removing it altogether.

4. We can see here that there are 4372 Unique Customers in this dataset.


5. Finally, we notice the presence of 5268 duplicate entries, and we have to remove those, to avoid distorting the model

In [9]:
data.df.dropna(inplace=True)
print('Null Values after handling:') 
data.df.isnull().sum()

Null Values after handling:


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [10]:
data.df.drop_duplicates(subset=None, keep='first', inplace=True)
data.df.duplicated().sum()

0

In [11]:
data.df = data.df[data.df['Quantity'] > 0]
print(data.df['Quantity'].min())
data.df = data.df[data.df['UnitPrice'] > 0]
print(data.df['UnitPrice'].min())

1
0.001


In [12]:
data.df.shape

(392692, 8)

In [13]:
total_orders = data.df.groupby(['CustomerID'])['InvoiceNo'].nunique()
multiple_orders = np.sum(total_orders > 1) / data.df['CustomerID'].nunique()
print(f'{100 * multiple_orders:.2f}% of customers ordered more than once.')

65.58% of customers ordered more than once.


In [26]:
data.df['TotalPrice'] = data.df['Quantity'] * data.df['UnitPrice']
data.df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [27]:
retail_one = data.df.loc[:, ['CustomerID', 'InvoiceNo','InvoiceDate','TotalPrice']]

In [28]:
retail_one['OrderPeriod'] = retail_one.InvoiceDate.apply(lambda x: x.strftime('%Y-%m'))
retail_one.head()

,CustomerID,InvoiceNo,InvoiceDate,TotalPrice,OrderPeriod
0,17850.0,536365,2010-12-01 08:26:00,15.30,2010-12
1,17850.0,536365,2010-12-01 08:26:00,20.34,2010-12
2,17850.0,536365,2010-12-01 08:26:00,22.00,2010-12
3,17850.0,536365,2010-12-01 08:26:00,20.34,2010-12
4,17850.0,536365,2010-12-01 08:26:00,20.34,2010-12


In [29]:
retail_one.set_index('CustomerID', inplace=True)

retail_one['CohortGroup'] = retail_one.groupby(level=0)['InvoiceDate'].min().apply(lambda x: x.strftime('%Y-%m'))
retail_one.reset_index(inplace=True)
retail_one.head()

,CustomerID,InvoiceNo,InvoiceDate,TotalPrice,OrderPeriod,CohortGroup
0,17850.0,536365,2010-12-01 08:26:00,15.30,2010-12,2010-12
1,17850.0,536365,2010-12-01 08:26:00,20.34,2010-12,2010-12
2,17850.0,536365,2010-12-01 08:26:00,22.00,2010-12,2010-12
3,17850.0,536365,2010-12-01 08:26:00,20.34,2010-12,2010-12
4,17850.0,536365,2010-12-01 08:26:00,20.34,2010-12,2010-12


In [30]:
retail_one['CohortGroup'].unique()

array(['2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05',
       '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11',
       '2011-12'], dtype=object)

In [31]:
retail_one['OrderPeriod'].unique()

array(['2010-12', '2011-01', '2011-02', '2011-03', '2011-04', '2011-05',
       '2011-06', '2011-07', '2011-08', '2011-09', '2011-10', '2011-11',
       '2011-12'], dtype=object)

In [48]:
grouped = retail_one.groupby(['CohortGroup','OrderPeriod'])

cohorts = grouped.agg({'CustomerID': pd.Series.nunique,
                      'InvoiceNo': pd.Series.nunique,
                      'TotalPrice': np.sum})

#rename columns to make them more meaningful
cohorts.rename(columns={'CustomerID': 'Customers',
                       'InvoiceNo': 'Orders'}, inplace=True)

cohorts.head()

Customers  Orders  TotalPrice
CohortGroup OrderPeriod                               
2010-12     2010-12          885.0    1400   570422.73
            2011-01          324.0     517   275734.47
            2011-02          286.0     458   233390.47
            2011-03          340.0     582   302367.44
            2011-04          321.0     504   204034.38

In [52]:
cohorts['CohortPeriod'] = np.arange(len(cohorts)) + 1

In [53]:
cohorts.head()

Customers  Orders  TotalPrice  CohortPeriod
CohortGroup OrderPeriod                                             
2010-12     2010-12          885.0    1400   570422.73             1
            2011-01          324.0     517   275734.47             2
            2011-02          286.0     458   233390.47             3
            2011-03          340.0     582   302367.44             4
            2011-04          321.0     504   204034.38             5

In [54]:
cohorts.shape

(91, 4)

In [56]:
# reindex the DataFrame
cohorts.reset_index(inplace=True)
cohorts.set_index(['CohortGroup', 'CohortPeriod'], inplace=True)

# create a Series holding the total size of each CohortGroup
cohort_group_size = cohorts['Customers'].groupby(level=0).first()
cohort_group_size.head()

CohortGroup
2010-12    885.0
2011-01    417.0
2011-02    380.0
2011-03    452.0
2011-04    300.0
Name: Customers, dtype: float64

In [59]:
cohorts.head()

OrderPeriod  Customers  Orders  TotalPrice
CohortGroup CohortPeriod                                           
2010-12     1                2010-12      885.0    1400   570422.73
            2                2011-01      324.0     517   275734.47
            3                2011-02      286.0     458   233390.47
            4                2011-03      340.0     582   302367.44
            5                2011-04      321.0     504   204034.38

In [67]:
cohorts["Customers"].unstack(0).head()

CohortGroup,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
CohortPeriod,,,,,,,,,,,,,
1,885.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,324.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,286.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,340.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,321.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
